In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os, glob
from sklearn.model_selection import train_test_split

# 이미지를 28x28 사이즈로 resizing 해 주는 함수
def resize_images(img_path):    
    
    images=glob.glob(img_path + "/*.jpg")  
    
    print(len(images), " images to be resized.")
    
    target_size=(28,28)
    for img in images:
        old_img=Image.open(img)
        new_img=old_img.resize(target_size,Image.ANTIALIAS)
        new_img.save(img, "JPEG")
    
    print(len(images), " images resized.")

In [2]:
# 가위바위보 이미지를 각각 구분해 주는 함수
def load_data(img_path, number_of_data=600):  
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    return imgs, labels, idx  # 수행 된 train과 test의 갯수를 idx로 함께 반환

In [4]:
# rock, paper, sissor 별로 각각의 경로를 지정
image_rock_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock"
image_paper_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/paper"
image_scissor_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/scissor"
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/"
image_rock_test_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/rock"
image_scissor_test_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/scissor"
image_paper_test_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/paper"
image_test_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"

# train과 test에 필요한 rock, paper, sissor 이미지들을 resizing 하기 위한 함수 호출
resize_images(image_rock_path)     # rock images for training
resize_images(image_paper_path)    # paper images for training
resize_images(image_scissor_path)  # sissor images for training
resize_images(image_rock_test_path)      # rock images for testing
resize_images(image_scissor_test_path)   # scissor images for testing
resize_images(image_paper_test_path)     # paper images for testing

200  images to be resized.
200  images resized.
200  images to be resized.
200  images resized.
200  images to be resized.
200  images resized.
100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.


In [7]:
# train과 test가 완료된 결과값 및 갯수를 return 받음
(x_train, y_train, idx)=load_data(image_dir_path)
print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
(x_test, y_test, idx)=load_data(image_test_path)
print("TEST용 데이터(x_test)의 이미지 개수는", idx,"입니다.")

학습데이터(x_train)의 이미지 개수는 600 입니다.
TEST용 데이터(x_test)의 이미지 개수는 300 입니다.


In [12]:
# Hyperparameters 값 설정
n_channel_1 = 16   # 이미지에서 분석하고자하는 특징의 갯수를 지정
n_channel_2 = 32
n_dense=128        # 분류에 사용되는 뉴런의 숫자. 2의 배수로 조절해가며 결과값을 비교
n_train_epoch=10   # 오차를 줄이기 위한 반복수행 횟수

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()  # 모델의 전반적인 정보를 출력

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               102528    
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [13]:
# 사용된 이미지의 Normalization(정규화) 진행. pixel의 경우 255.0으로 나눠줌.
x_train_norm = x_train / 255.0
x_test_norm = x_test / 255.0

In [14]:
# 칼라 이미지를 사용하기 위해 3 (RGB에 해당)을  argument로 입력. reshaping 진행
x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 3)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산
x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 3)

In [18]:
# keras model을 compile 하기 위해 필요한 optimizer, loss, metrics를 입력
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# 지정된 횟수만큼 training을 진행하여 accuracy를 확인
model.fit(x_train_reshaped, y_train, epochs=n_train_epoch)

Epoch 1/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0974 - accuracy: 0.9698
Epoch 2/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0154 - accuracy: 1.0000
Epoch 3/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0109 - accuracy: 1.0000
Epoch 4/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0073 - accuracy: 1.0000
Epoch 5/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0054 - accuracy: 1.0000
Epoch 6/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 7/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 8/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 9/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 10/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0023 - accuracy: 1.0000


In [19]:
# Training이 된 model을 이용해 실제 test를 진행
test_loss, test_accuracy = model.evaluate(x_test_reshaped, y_test, verbose=2)
print("test_loss: {} ".format(test_loss)) # Test 결과 오차값을 보여줌
print("test_accuracy: {}".format(test_accuracy)) # Test 결과 accuracy 값을 보여줌

19/19 - 0s - loss: 2.6010 - accuracy: 0.8083
test_loss: 2.600996494293213 
test_accuracy: 0.8083333373069763


**최종적으로 80% 이상의 accuracy를 가지는 결과를 얻음.**

**회고**
: 최종 결과값인 accuracy를 높이기 위해 hyperparameter들을 하나씩 조절해 본 결과 dense를 128로 하고 epoch는 10으로 했을 때 지금과같은 0.8 이상의 accuracy를 얻을 수 있었다.
처음에 300장의 image만을 가지고 training을 진행한 뒤, 다른사람이 공유해준 300장의 image를 이용해 test 해봤을 때는 최대 0.6이 조금 넘는 정도의 accuracy를 보였다. 따라서 300장을 추가적으로 training 시킨뒤 다시 결과값을 도출해 봤을때 0.8 이상의 accuracy를 얻을 수 있었다. 이미지를 training 시킬 때 여러 각도, 조명, 배경 등의 요소들을 함께 training 시켜서 추후에 test 에 사용되는 이미지들의 outlier를 최소화 시키는게 무엇보다 중요하다고 생각한다.